<a href="https://colab.research.google.com/github/staaason/Labs_crypto/blob/main/lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Завдання 1** Реалiзуйте алгоритм знаходження твiрного елемента a мультиплiкативної групи
поля GF(p), p — непарне просте, бiнарна довжина якого не бiльша за 20.
Оптимізована ідея була взята з [Finding a primitive root of a prime number](https://math.stackexchange.com/questions/124408/finding-a-primitive-root-of-a-prime-number/133720#133720)

In [5]:
import math
import random

def check_prime(n):
    is_prime = [1] * (n + 1)
    is_prime[0] = is_prime[1] = 0
    i = 2
    while i * i <= n:
        if is_prime[i] == 1:
            j = i * i
            while j <= n:
                is_prime[j] = 0
                j += i
        i += 1
    return is_prime[n] == 1


def generate_prime():
    while True:
        p = random.randint(1, 2**20 - 1)
        if p % 2 == 0:
            p += 1
        if check_prime(p):
            return p
class GF:
    def __init__(self, p):
        if p % 2 == 0 and not check_prime(p):
            raise ValueError("Number is odd and not prime")
        self.p = p
        self.n = p - 1
        self.elements = [i for i in range(0, self.n + 1)]


    def prime_factors(self, n):
        factors = []
        while n % 2 == 0:
            factors.append(2)
            n //= 2
        for i in range(3, int(n ** 0.5) + 1, 2):
            while n % i == 0:
                factors.append(i)
                n //= i
        if n > 2:
            factors.append(n)
        return factors

    def find_primitives(self):
        primitives = set()
        prime_factors = self.prime_factors(self.n)
        for i in range(1, self.n + 1):
            is_primitive = True
            for j in prime_factors:
                if pow(i, self.n // j, self.p) == 1:
                    is_primitive = False
                    break
            if is_primitive:
                primitives.add(i)
        return primitives

    def baby_step_giant_step(self, alpha, beta):
        table = dict()
        m = math.ceil(math.sqrt(self.n))
        for j in range(0, m):
            table[pow(alpha, j, self.p)] = j
        a_m = pow(alpha, m * (self.p - 2), self.p)
        y = beta
        for i in range(0, m):
            if y in table:
                return i * m + table[y]
            else:
                y = (y * a_m) % self.p


p = generate_prime()
gf = GF(p)
print(f'p = {p}')
primitives = gf.find_primitives()
print(len(primitives))
print(primitives)


p = 999541
266112
{524288, 2, 524290, 524291, 6, 524295, 10, 524301, 524303, 22, 23, 24, 524310, 524311, 524316, 29, 524317, 524319, 37, 524328, 524331, 524333, 524334, 47, 53, 56, 57, 524344, 524349, 524352, 66, 69, 70, 72, 524360, 524362, 524363, 524366, 79, 82, 83, 86, 87, 90, 524378, 95, 524383, 98, 524388, 524389, 102, 524390, 104, 107, 524395, 524397, 110, 111, 524401, 115, 120, 524409, 126, 127, 128, 524414, 130, 524415, 141, 142, 145, 524436, 150, 524441, 524444, 157, 159, 162, 524450, 167, 170, 524460, 173, 524461, 524462, 524465, 178, 524466, 182, 524472, 185, 524477, 191, 524481, 194, 524482, 524483, 524485, 524487, 200, 524488, 202, 524492, 524495, 210, 524499, 524500, 524505, 524509, 224, 226, 227, 524517, 524521, 234, 235, 237, 239, 242, 524530, 524531, 524532, 248, 249, 250, 524539, 253, 524545, 524547, 263, 264, 265, 524551, 524552, 524553, 524554, 270, 271, 524555, 524561, 274, 524556, 276, 277, 278, 281, 524569, 283, 524571, 524574, 288, 524577, 293, 524581, 524582, 5

**Завдання 2.**Реалiзуйте алгоритм малих i великих крокiв розв’язування задачi дискретного ло-
гарифма в групi G = ⟨a⟩.

In [16]:
gf_ = GF(11)
print(f'p = {11}, g = {3}, h = {7}: logh = {gf_.baby_step_giant_step(3, 7) if gf_.baby_step_giant_step(3, 7) is not None else "Не визначений"}')

gf_2 = GF(23)
print(f'p = {23}, g = {2}, h = {13}: logh = {gf_2.baby_step_giant_step(2, 13)}')

gf_3 = GF(71)
print(f'p = {71}, g = {11}, h = {21}: logh = {gf_3.baby_step_giant_step(11, 21)}')

gf_4 = GF(47)
print(f'p = {47}, g = {10}, h = {22}: logh = {gf_4.baby_step_giant_step(10, 22)}')

gf_5 = GF(941)
print(f'p = {941}, g = {627}, h = {608}: logh = {gf_5.baby_step_giant_step(627, 608)}')




p = 11, g = 3, h = 7: logh = Не визначений
p = 23, g = 2, h = 13: logh = 7
p = 71, g = 11, h = 21: logh = 37
p = 47, g = 10, h = 22: logh = 11
p = 941, g = 627, h = 608: logh = 18


**Завдання 3.** Реалiзуйте протокол Дiффi-Хеллмана на основi групи G.

In [18]:
class Diffie_Hellman:

    def gen_secret(self, p, g, a, b):
        gf = GF(p)
        primitives = gf.find_primitives()
        if g not in primitives:
            raise ValueError('g is not primitive root')
        public_key_a = pow(g, a, p)
        print(f'x = {public_key_a}')
        public_key_b = pow(g, b, p)
        print(f'y = {public_key_b}')
        shared_secret_key_1 = pow(public_key_b, a, p)
        shared_secret_key_2 = pow(public_key_a, b, p)
        print(f'Shared secret key = {shared_secret_key_1}')
        print(shared_secret_key_1 == shared_secret_key_2)
        print(f'a = {gf.baby_step_giant_step(g, public_key_a)}')
        print(f'b = {gf.baby_step_giant_step(g, public_key_b)}')
        return public_key_a, public_key_b, shared_secret_key_1

random_primitive = random.choice(list(primitives))
print(f'<{random_primitive}>')
a = random.randint(3, p - 2)
b = random.randint(3, p - 2)
df = Diffie_Hellman()
print(f'a = {a}, b={b}')
df.gen_secret(p, random_primitive, a, b)

<274384>
a = 265679, b=600181
x = 54974
y = 43724
Shared secret key = 468983
True
a = 265679
b = 600181


(54974, 43724, 468983)